In [1]:
print("""
@File         : CH08.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-09-02 12:14:15
@Email        : cuixuanstephen@gmail.com
@Description  : 使用读写包操作 Excel 文件
""")


@File         : CH08.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-09-02 12:14:15
@Email        : cuixuanstephen@gmail.com
@Description  : 使用读写包操作 Excel 文件



In [2]:
%cd ../

d:\Data-Analysis-and-Science\PY4XL


## 读写包

### 何时使用何种包

|Excel文件格式|读|写|编辑|
|---|---|---|---|
|xlsx|OpenPyXL|OpenPyXL, XlsxWriter|OpenPyXL|
|xlsm|OpenPyXL|OpenPyXL, XlsxWriter|OpenPyXL|
|xltx, xltm|OpenPyXL|OpenPyXL|OpenPyXL|
|xlsb|pyxlsb|-|-|
|xls, xlt|xlrd|xlwt|xlutils

pandas 会使用它可以找到的读取包，如果同时安装了 `OpenPyXL` 和 `XlsxWriter`，那么 pandas 默认使用 `XlsxWriter`。如果你想亲自选择 pandas 所使用的包，则可以在 `read_excel` 或 `to_excel`，以及 `ExcelFile` 或 `ExcelWriter` 的 `engine` 参数中指定所选包。`engine`（引擎）是小写的包名，因此如果要用 `OpenPyXL` 而不是 `XlsxWriter` 来写文件，则需要执行如下代码：

```python
df.to_excel("filename.xlsx", engine="openpyxl")
```

In [3]:
import pandas as pd
import sys
from pathlib import Path

sys.path.append(Path('.').resolve() / 'sources')

import openpyxl
import sources.excel as excel
import datetime as dt

### OpenPyXL

要获得单元格的值，需要使用 `data_only=True` 参数来打开工作簿，其默认值是 `False`，此时会返回单元格的公式而不是值：

In [4]:
# 在加载数据之后文件会自动关闭
book = openpyxl.load_workbook('xl/stores.xlsx', data_only=True)

# 通过名称或索引（从0开始）获取工作表对象
sheet = book['2019']
sheet = book.worksheets[0]

In [5]:
book.sheetnames

['2019', '2020', '2019-2020']

In [7]:
for i in book.worksheets:
    print(i.title)
    
# 遍历所有工作表对象
# openpyxl 使用的是 title 而不是 name

2019
2020
2019-2020


In [9]:
# 获取维度，以工作表
# 所选区域为例
sheet.max_row, sheet.max_column

(8, 6)

In [11]:
# 读取单个单元格的值，分别使用的是 A1 这种
# 表示法，以及单元格索引（从 1 开始）

sheet['B6'].value

'Boston'

In [12]:
sheet.cell(row=6, column=2).value

'Boston'

In [14]:
# 作者自定义函数
data = excel.read(book['2019'], (2, 2), (8, 6))
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

#### 使用 OpenPyXL 写入文件

OpenPyXL 会在内存中构建 Excel 文件，当你调用 `save` 方法时会将其写入文件。

In [15]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
from sources import excel

In [29]:
book = openpyxl.Workbook()

sheet = book.active
sheet.title = 'Sheet1'

In [30]:
sheet['A1'].value = "Hello 1"
sheet.cell(row=2, column=1, value='Hell0, 2')

<Cell 'Sheet1'.A2>

In [31]:
# 格式化：填充颜色、对齐、边框和字体
font_format = Font(color='FF0000', bold=True)
thin = Side(border_style='thin', color='FF0000')
sheet['A3'].value = 'Hello, 3'
sheet['A3'].font = font_format
sheet['A3'].border = Border(top=thin, left=thin, right=thin, bottom=thin)
sheet['A3'].alignment = Alignment(horizontal='center')
sheet['A3'].fill = PatternFill(fgColor='FFFF00', fill_type='solid')

In [32]:
# 数字格式化（使用 Excel 的格式化字符串）
sheet['A4'].value = 3.3333
sheet['A4'].number_format = "0.00"

In [33]:
# 日期格式化（使用 Excel 的格式化字符串）
sheet['A5'].value = dt.date(2016, 10, 13)
sheet['A5'].number_format = 'mm/dd/yy'

In [34]:
# 公式：必须使用以逗号分隔的英文公式名称
sheet['A6'].value = '=SUM(A4, 2)'

In [35]:
sheet.add_image(Image('images/python.png'), 'C1')
# 图片

In [36]:
data = [[None, 'North', 'South'],
        ['Last Year', 2, 5],
        ['This Year', 3, 6],]

excel.write(sheet, data, 'A10')

In [37]:
chart = BarChart()
chart.type = 'col'
chart.title = 'Sales Per Region'
chart.x_axis.title = 'Regions'
chart.y_axis.title = 'Sales'
chart_data = Reference(sheet, min_row=11, min_col=1, max_row=12, max_col=3)

chart_categories = Reference(sheet, min_row=10, min_col=2, max_row=10, max_col=3)

In [38]:
chart.add_data(chart_data, titles_from_data=True, from_rows=True)
chart.set_categories(chart_categories)

sheet.add_chart(chart, 'A15')

In [39]:
book.save('res/openpyxl.xlsx')

如果想写入为 Excel 模板文件，那么需要在保存前设置 `template` 属性为 True：

In [40]:
book = openpyxl.Workbook()
sheet = book.active

sheet['A1'].value = 'This is a template'
book.template = True
book.save('res/template.xltx')

#### 使用 OpenPyXL 编辑文件

并不存在真正可以编辑 Excel 文件的读写包：实际上 OpenPyXL 首先会读取所有它可以理解的数据，然后再将文件数据从头到尾写回去，其间你做出的所有更改也会包含其中。对于一些主要包含已格式化的单元格以及数据和公式的简单 Excel 文件来说，它的功能已经非常强大了。但如果你的工作表中包含图表或者其他高级内容，那么 OpenPyXL 的功能就显得非常有限了，这些内容要么会被修改，要么会被直接丢弃。例如，在 OpenPyXL v3.0.5 版本中，图表会被重命名且标题会被丢弃。下面是编辑 Excel 文件的示例：

In [41]:
book = openpyxl.load_workbook('xl/stores.xlsx')
book['2019']['A1'].value = 'modified'
book.save('res/stores_edited.xlsx')

如果想写入为 xlsm 文件，那么 OpenPyXL 就必须处理一个已经存在的文件，并且在加载时需要将 `keep_vba` 参数设置为 `True`：

In [42]:
book = openpyxl.load_workbook('xl/macro.xlsm', keep_vba=True)
book['Sheet1']['A1'].value = 'Click the button!'
book.save('res/macro_openpyxl.xlsm')

### XlsxWriter

顾名思义，XlsxWriter 只能写入 Excel 文件注意，XlsxWriter 使用的是从 0 开始的单元格索引，
而 OpenPyXL 使用的是从 1 开始的单元格索引，一定要在切换两个包时考虑到这一点。

In [44]:
import xlsxwriter

In [46]:
# 实例化工作簿
book = xlsxwriter.Workbook('res/xlsxwriter.xlsx')

In [47]:
# 添加工作表并为其命名
sheet = book.add_worksheet('Sheet1')

### pyxlsb

和其他读取库相比，pyxlsb 提供的功能不多，但是如果你要读取二进制的 xlsb 格式的 Excel 文件，那么 pyxlsb 就成了唯一选择。